<a href="https://colab.research.google.com/github/alexsanderthorne/BigDataCorpTest/blob/main/intuitive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. TESTE DE WEB SCRAPING

1. Install necessary libraries



2. Accessing the website and downloading the PDFs

In [4]:
import requests
from bs4 import BeautifulSoup
import zipfile

# URL of the website
url = "https://www.gov.br/ans/pt-br/acesso-a-informacao/participacao-da-sociedade/atualizacao-do-rol-de-procedimentos"

# Send a request to the website and get the HTML content
response = requests.get(url)
html_content = response.content

# Parse the HTML content using BeautifulSoup
soup = BeautifulSoup(html_content, "html.parser")

# Find all the links on the page
all_links = soup.find_all("a")

# Filter for links to PDF files for Anexo I and Anexo II
pdf_links = [
    link["href"]
    for link in all_links
    if link.get("href")
    and (link.text.strip().startswith("Anexo I") or link.text.strip().startswith("Anexo II"))
    and link["href"].endswith(".pdf")
]

# Download the PDF files and store their filenames
downloaded_filenames = []
for link in pdf_links:
    file_name = link.split("/")[-1]
    downloaded_filenames.append(file_name)  # Store the filename
    print(f"Downloading {file_name}...")
    pdf_response = requests.get(link)
    with open(file_name, "wb") as f:
        f.write(pdf_response.content)
    print(f"{file_name} downloaded successfully!")

# Compress the PDF files into a ZIP archive using stored filenames
with zipfile.ZipFile("Alexsander.zip", "w") as zipf:
    for file_name in downloaded_filenames:  # Use stored filenames
        zipf.write(file_name)

print("Anexos compressed successfully!")

Anexo_I_Rol_2021RN_465.2021_RN627L.2024.pdf downloaded successfully!
Anexo_II_DUT_2021_RN_465.2021_RN628.2025_RN629.2025.pdf downloaded successfully!
Anexos compressed successfully!


Install necessary libraries

2. TESTE DE TRANSFORMAÇÃO DE DADOS


In [5]:
!pip install PyPDF2 tabula-py

In [6]:
!pip install PyPDF2 tabula-py gdown

from google.colab import drive
drive.mount('/content/drive')

import PyPDF2
import tabula
import pandas as pd
import zipfile
import gdown

# 1. Download the PDF file from the Google Drive link using gdown
file_url = 'https://drive.google.com/file/d/1-vPNsPLBLy1eDbUZxLhCnmmco4Z6i5fe/view?usp=drive_link'
file_id = file_url.split('/')[-2] # Extract the file ID from the URL
gdown.download(id=file_id, output='downloaded_file.pdf', quiet=False)


# 2. Extrair dados do PDF
tables = tabula.read_pdf('downloaded_file.pdf', pages="all", multiple_tables=True)

# Combinando todas as tabelas em um único DataFrame
df = pd.concat(tables)

# 3. Salvar dados em CSV
df.to_csv("rol_procedimentos.csv", index=False)

# 4. Compactar o CSV
with zipfile.ZipFile("Alexsander.zip", "w") as zipf:
    zipf.write("rol_procedimentos.csv")

# 5. Substituir abreviações
# Substituindo OD por "Odontologia" e AMB por "Ambulatorial"
df = df.replace({"OD": "Odontologia", "AMB": "Ambulatorial"})

# Salvar o DataFrame com as abreviações substituídas
df.to_csv("rol_procedimentos_final.csv", index=False)

print("Processo concluído com sucesso!")

Mounted at /content/drive


Downloading...
From: https://drive.google.com/uc?id=1-vPNsPLBLy1eDbUZxLhCnmmco4Z6i5fe
To: /content/downloaded_file.pdf
100%|██████████| 13.7M/13.7M [00:00<00:00, 85.5MB/s]
Apr 04, 2025 12:21:26 PM org.apache.pdfbox.pdmodel.font.FileSystemFontProvider <init>
Apr 04, 2025 12:21:26 PM org.apache.pdfbox.pdmodel.font.FileSystemFontProvider <init>



Processo concluído com sucesso!


3. TESTE DE BANCO DE DADOS

Tarefas de Preparação:

In [9]:
import ftplib
import os

ftp_server = 'dadosabertos.ans.gov.br'
ftp_path = '/FTP/PDA/demonstracoes_contabeis/'

ftp = ftplib.FTP(ftp_server)
ftp.login()  # Use ftp.login('username', 'password') if authentication is required
ftp.cwd(ftp_path)

def download_files(ftp, current_path):
    """Recursively downloads files from the given FTP path."""
    file_list = ftp.nlst()  # Get the list of files and directories

    for item in file_list:
        item_path = os.path.join(current_path, item)
        print(f"Found item: {item_path}")

        try:
            # Try to get file size; raises error for directories
            ftp.size(item)
            # It's a file, so download it
            local_file_path = os.path.join("demonstracoes_contabeis", item_path)
            os.makedirs(os.path.dirname(local_file_path), exist_ok=True)
            with open(local_file_path, 'wb') as local_file:
                ftp.retrbinary('RETR ' + item, local_file.write)
            print(f"Downloaded: {item_path}")
        except ftplib.error_perm as e:
            if str(e).startswith('550'):  # Likely a directory
                print(f"Entering directory: {item_path}")
                ftp.cwd(item)  # Change to the subdirectory
                download_files(ftp, item_path)  # Recursively download files inside
                ftp.cwd("..")  # Go back up one level
                print(f"Exiting directory: {item_path}")
            else:
                print(f"Skipping {item_path} (permission error): {e}")

# Start the download process from the base path
download_files(ftp, '')

ftp.quit()



# --- Downloading Operadoras Ativas ---
operadoras_url = "https://dadosabertos.ans.gov.br/FTP/PDA/operadoras_de_plano_de_saude_ativas/"
operadoras_path = "Relatorio_cadop.csv"
try:
    with open(operadoras_path, 'wb') as f:
        response = requests.get(operadoras_url)
        response.raise_for_status()  # Raise HTTPError for bad responses (4xx or 5xx)
        f.write(response.content)
    print("Downloaded: CadOp.csv")

except requests.exceptions.RequestException as e:
    print(f"Error downloading CadOp.csv: {e}")

Found item: 2007
Entering directory: 2007
Found item: 2007/2007_1_trimestre.zip
Downloaded: 2007/2007_1_trimestre.zip
Found item: 2007/2007_2_trimestre.zip
Downloaded: 2007/2007_2_trimestre.zip
Found item: 2007/2007_3_trimestre.zip
Downloaded: 2007/2007_3_trimestre.zip
Found item: 2007/2007_4_trimestre.zip
Downloaded: 2007/2007_4_trimestre.zip
Exiting directory: 2007
Found item: 2008
Entering directory: 2008
Found item: 2008/2008_1_trimestre.zip
Downloaded: 2008/2008_1_trimestre.zip
Found item: 2008/2008_2_trimestre.zip
Downloaded: 2008/2008_2_trimestre.zip
Found item: 2008/2008_3_trimestre.zip
Downloaded: 2008/2008_3_trimestre.zip
Found item: 2008/2008_4_trimestre.zip
Downloaded: 2008/2008_4_trimestre.zip
Exiting directory: 2008
Found item: 2009
Entering directory: 2009
Found item: 2009/2009_1_trimestre.zip
Downloaded: 2009/2009_1_trimestre.zip
Found item: 2009/2009_2_trimestre.zip
Downloaded: 2009/2009_2_trimestre.zip
Found item: 2009/2009_3_trimestre.zip
Downloaded: 2009/2009_3_trim